In [2]:
import os
import fnmatch


def filecount(dir_path):
    co= len(fnmatch.filter(os.listdir(dir_path), '*.*'))
    print(co)
    return co

dir1=r'C:\Users\hp\Assignment\Artificially-Blurred'
count_art=filecount(dir1)

dir2=r'C:\Users\hp\Assignment\Naturally-Blurred'
count_nat=filecount(dir2)

dir3=r'C:\Users\hp\Assignment\Undistorted'
count_undi=filecount(dir3)

150
220
630


In [6]:
import cv2
from os import listdir
import numpy as np

In [7]:
from PIL import Image

def is_image_corrupted(f_path):
    for name in listdir(f_path):
        file_path=os.path.join(f_path, name)
    try:
        with Image.open(file_path) as img:
            img.verify()
            print("not corrupted")
    except:
        print("corrupted")
is_image_corrupted(dir1)
is_image_corrupted(dir2)
is_image_corrupted(dir3)

not corrupted
not corrupted
not corrupted


In [8]:
def get_data(f_path):
    features=[]
    target_size = (256, 256)
    for name in listdir(f_path):
        feature=[]
        f_name=os.path.join(f_path, name)
        #print(f_name)
        img_gray = cv2.imread(f_name, cv2.IMREAD_GRAYSCALE)
        if img_gray is not None:
            img_gray = cv2.resize(img_gray, target_size)
            try:
                lap_feat = cv2.Laplacian(img_gray , cv2.CV_64F)
                #print(lap_feat.shape)
                sobelx = cv2.Sobel(img_gray,cv2.CV_64F,1,0,ksize=3)
                sobely = cv2.Sobel(img_gray,cv2.CV_64F,0,1,ksize=3)
                sob_feat = cv2.sqrt(cv2.addWeighted(cv2.pow(sobelx,2.0), 1.0, cv2.pow(sobely,2.0), 1.0, 0.0))
                feature.extend([f_name,lap_feat.mean(),lap_feat.var(),
                            sob_feat.mean(),sob_feat.var()])
                if feature:
                    features.append(feature)
            except:
                print("Error occurred while computing features for image: ", f_name)
        else:
            print("Error: Image is empty or null: ", f_name)
    
    return features


fe_art=get_data(dir1)
fe_nat=get_data(dir2)
fe_und=get_data(dir3)

Error: Image is empty or null:  C:\Users\hp\Assignment\Undistorted\╬ò╬╣╬║╬┐╠ü╬╜╬▒_00...jpg
Error: Image is empty or null:  C:\Users\hp\Assignment\Undistorted\╬ò╬╣╬║╬┐╠ü╬╜╬▒_001...jpg
Error: Image is empty or null:  C:\Users\hp\Assignment\Undistorted\╬ò╬╣╬║╬┐╠ü╬╜╬▒_002...jpg
Error: Image is empty or null:  C:\Users\hp\Assignment\Undistorted\╬ò╬╣╬║╬┐╠ü╬╜╬▒_003...jpg
Error: Image is empty or null:  C:\Users\hp\Assignment\Undistorted\╬ò╬╣╬║╬┐╠ü╬╜╬▒_004...jpg


In [10]:
print(np.shape(fe_art))
print(np.shape(fe_nat))
print(np.shape(fe_und))

(150, 5)
(220, 5)
(625, 5)


In [11]:
import pandas as pd
d1 = pd.DataFrame(fe_art)
d2 = pd.DataFrame(fe_nat)
d3 = pd.DataFrame(fe_und)

In [12]:
df = pd.concat([d1, d2, d3], axis=0)
df.drop(0,axis=1,inplace=True)
df.head()

,1,2,3,4
0,0.191391,13.529944,12.299625,520.930923
1,0.274292,32.033712,30.416960,1524.161106
2,0.278992,38.186690,38.992206,1679.026837
3,0.274445,35.872831,38.494882,1250.533017
4,0.229507,26.083725,26.404438,1219.115580


In [13]:
df.columns = ['Lap_mean', 'Lap_var','sob_mean', 'sob_var']

In [14]:
t = [0] * 150 + [1] * 220 + [2] * 625
df['Blur_type']=t
df.head()

,Lap_mean,Lap_var,sob_mean,sob_var,Blur_type
0,0.191391,13.529944,12.299625,520.930923,0
1,0.274292,32.033712,30.416960,1524.161106,0
2,0.278992,38.186690,38.992206,1679.026837,0
3,0.274445,35.872831,38.494882,1250.533017,0
4,0.229507,26.083725,26.404438,1219.115580,0


In [15]:
x =df.iloc[:,:-1].values.astype("float")
y=df.iloc[:,-1].values

In [16]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
from keras.utils import np_utils

def data_preprocess(x,y):
    dummy_y = np_utils.to_categorical(y)
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)
    return x_scaled,dummy_y

In [18]:
x_scaled,dummy_y=data_preprocess(x,y)

In [19]:
print(x_scaled[:3])
print(dummy_y[:3])

[[ 1.23295511 -0.93606873 -1.75868799 -1.34151283]
 [ 1.90895896 -0.9308289  -1.24127647 -1.19122742]
 [ 1.94728202 -0.92908652 -0.9963767  -1.1680283 ]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, KFold

def baseline_model():
    model = Sequential()
    model.add(Dense(8, input_dim=4, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=100, batch_size=32, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)
estimator.fit(x_scaled, dummy_y)
estimator.model.save('pretrained_model.h5')

results = cross_val_score(estimator, x_scaled, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/100


C:\Users\hp\AppData\Local\Temp\ipykernel_3232\2744962095.py:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=baseline_model, epochs=100, batch_size=32, verbose=1)


32/32 [==============================] - 0s 1ms/step - loss: 1.0825 - accuracy: 0.4462
Epoch 2/100
32/32 [==============================] - 0s 962us/step - loss: 0.9558 - accuracy: 0.6633
Epoch 3/100
32/32 [==============================] - 0s 949us/step - loss: 0.8564 - accuracy: 0.7035
Epoch 4/100
32/32 [==============================] - 0s 856us/step - loss: 0.7864 - accuracy: 0.7055
Epoch 5/100
32/32 [==============================] - 0s 973us/step - loss: 0.7354 - accuracy: 0.7176
Epoch 6/100
32/32 [==============================] - 0s 990us/step - loss: 0.6975 - accuracy: 0.7146
Epoch 7/100
32/32 [==============================] - 0s 901us/step - loss: 0.6691 - accuracy: 0.7246
Epoch 8/100
32/32 [==============================] - 0s 965us/step - loss: 0.6561 - accuracy: 0.7236
Epoch 9/100
32/32 [==============================] - 0s 849us/step - loss: 0.6360 - accuracy: 0.7286
Epoch 10/100
32/32 [==============================] - 0s 1ms/step - loss: 0.6135 - accuracy: 0.7397
Epoch

32/32 [==============================] - 0s 929us/step - loss: 0.4217 - accuracy: 0.8312
Epoch 82/100
32/32 [==============================] - 0s 911us/step - loss: 0.4360 - accuracy: 0.8161
Epoch 83/100
32/32 [==============================] - 0s 920us/step - loss: 0.4277 - accuracy: 0.8211
Epoch 84/100
32/32 [==============================] - 0s 895us/step - loss: 0.4264 - accuracy: 0.8211
Epoch 85/100
32/32 [==============================] - 0s 904us/step - loss: 0.4248 - accuracy: 0.8332
Epoch 86/100
32/32 [==============================] - 0s 842us/step - loss: 0.4280 - accuracy: 0.8281
Epoch 87/100
32/32 [==============================] - 0s 822us/step - loss: 0.4149 - accuracy: 0.8312
Epoch 88/100
32/32 [==============================] - 0s 834us/step - loss: 0.4290 - accuracy: 0.8251
Epoch 89/100
32/32 [==============================] - 0s 878us/step - loss: 0.4257 - accuracy: 0.8261
Epoch 90/100
32/32 [==============================] - 0s 892us/step - loss: 0.4192 - accuracy: 

28/28 [==============================] - 0s 869us/step - loss: 0.4504 - accuracy: 0.8067
Epoch 62/100
28/28 [==============================] - 0s 936us/step - loss: 0.4335 - accuracy: 0.8123
Epoch 63/100
28/28 [==============================] - 0s 781us/step - loss: 0.4468 - accuracy: 0.8123
Epoch 64/100
28/28 [==============================] - 0s 1ms/step - loss: 0.4441 - accuracy: 0.8067
Epoch 65/100
28/28 [==============================] - 0s 919us/step - loss: 0.4498 - accuracy: 0.8078
Epoch 66/100
28/28 [==============================] - 0s 699us/step - loss: 0.4374 - accuracy: 0.8212
Epoch 67/100
28/28 [==============================] - 0s 934us/step - loss: 0.4415 - accuracy: 0.8078
Epoch 68/100
28/28 [==============================] - 0s 759us/step - loss: 0.4338 - accuracy: 0.8279
Epoch 69/100
28/28 [==============================] - 0s 697us/step - loss: 0.4228 - accuracy: 0.8246
Epoch 70/100
28/28 [==============================] - 0s 1ms/step - loss: 0.4427 - accuracy: 0.80

...

28/28 [==============================] - 0s 944us/step - loss: 0.5862 - accuracy: 0.7545
Epoch 21/100
28/28 [==============================] - 0s 469us/step - loss: 0.5784 - accuracy: 0.7612
Epoch 22/100
28/28 [==============================] - 0s 858us/step - loss: 0.5798 - accuracy: 0.7478
Epoch 23/100
28/28 [==============================] - 0s 648us/step - loss: 0.5670 - accuracy: 0.7634
Epoch 24/100
28/28 [==============================] - 0s 833us/step - loss: 0.5589 - accuracy: 0.7634
Epoch 25/100
28/28 [==============================] - 0s 973us/step - loss: 0.5560 - accuracy: 0.7656
Epoch 26/100
28/28 [==============================] - 0s 840us/step - loss: 0.5504 - accuracy: 0.7656
Epoch 27/100
28/28 [==============================] - 0s 974us/step - loss: 0.5545 - accuracy: 0.7511
Epoch 28/100
28/28 [==============================] - 0s 787us/step - loss: 0.5398 - accuracy: 0.7712
Epoch 29/100
28/28 [==============================] - 0s 1ms/step - loss: 0.5383 - accuracy: 0.

4/4 [==============================] - 0s 5ms/step - loss: 0.3423 - accuracy: 0.8485
Epoch 1/100
28/28 [==============================] - 0s 1ms/step - loss: 0.9968 - accuracy: 0.5904
Epoch 2/100
28/28 [==============================] - 0s 946us/step - loss: 0.9183 - accuracy: 0.6585
Epoch 3/100
28/28 [==============================] - 0s 974us/step - loss: 0.8353 - accuracy: 0.7009
Epoch 4/100
28/28 [==============================] - 0s 1ms/step - loss: 0.7822 - accuracy: 0.7042
Epoch 5/100
28/28 [==============================] - 0s 973us/step - loss: 0.7169 - accuracy: 0.7254
Epoch 6/100
28/28 [==============================] - 0s 717us/step - loss: 0.6890 - accuracy: 0.7165
Epoch 7/100
28/28 [==============================] - 0s 1ms/step - loss: 0.6449 - accuracy: 0.7299
Epoch 8/100
28/28 [==============================] - 0s 745us/step - loss: 0.6311 - accuracy: 0.7221
Epoch 9/100
28/28 [==============================] - 0s 781us/step - loss: 0.6058 - accuracy: 0.7545
Epoch 10/100

Epoch 80/100
28/28 [==============================] - 0s 1ms/step - loss: 0.4505 - accuracy: 0.8125
Epoch 81/100
28/28 [==============================] - 0s 667us/step - loss: 0.4436 - accuracy: 0.8192
Epoch 82/100
28/28 [==============================] - 0s 698us/step - loss: 0.4537 - accuracy: 0.8225
Epoch 83/100
28/28 [==============================] - 0s 813us/step - loss: 0.4599 - accuracy: 0.8013
Epoch 84/100
28/28 [==============================] - 0s 726us/step - loss: 0.4470 - accuracy: 0.8147
Epoch 85/100
28/28 [==============================] - 0s 1ms/step - loss: 0.4676 - accuracy: 0.8002
Epoch 86/100
28/28 [==============================] - 0s 841us/step - loss: 0.4516 - accuracy: 0.8036
Epoch 87/100
28/28 [==============================] - 0s 924us/step - loss: 0.4546 - accuracy: 0.8069
Epoch 88/100
28/28 [==============================] - 0s 1ms/step - loss: 0.4557 - accuracy: 0.8069
Epoch 89/100
28/28 [==============================] - 0s 504us/step - loss: 0.4513 - acc

28/28 [==============================] - 0s 1ms/step - loss: 0.4420 - accuracy: 0.8248
Epoch 60/100
28/28 [==============================] - 0s 663us/step - loss: 0.4385 - accuracy: 0.8259
Epoch 61/100
28/28 [==============================] - 0s 714us/step - loss: 0.4387 - accuracy: 0.8192
Epoch 62/100
28/28 [==============================] - 0s 1ms/step - loss: 0.4348 - accuracy: 0.8237
Epoch 63/100
28/28 [==============================] - 0s 841us/step - loss: 0.4474 - accuracy: 0.8170
Epoch 64/100
28/28 [==============================] - 0s 1ms/step - loss: 0.4412 - accuracy: 0.8181
Epoch 65/100
28/28 [==============================] - 0s 839us/step - loss: 0.4294 - accuracy: 0.8326
Epoch 66/100
28/28 [==============================] - 0s 637us/step - loss: 0.4361 - accuracy: 0.8225
Epoch 67/100
28/28 [==============================] - 0s 912us/step - loss: 0.4537 - accuracy: 0.8047
Epoch 68/100
28/28 [==============================] - 0s 538us/step - loss: 0.4469 - accuracy: 0.8192

28/28 [==============================] - 0s 871us/step - loss: 0.4830 - accuracy: 0.7946
Epoch 40/100
28/28 [==============================] - 0s 1ms/step - loss: 0.4820 - accuracy: 0.7891
Epoch 41/100
28/28 [==============================] - 0s 602us/step - loss: 0.4886 - accuracy: 0.7946
Epoch 42/100
28/28 [==============================] - 0s 673us/step - loss: 0.4777 - accuracy: 0.7924
Epoch 43/100
28/28 [==============================] - 0s 1ms/step - loss: 0.4759 - accuracy: 0.8013
Epoch 44/100
28/28 [==============================] - 0s 871us/step - loss: 0.4800 - accuracy: 0.7913
Epoch 45/100
28/28 [==============================] - 0s 875us/step - loss: 0.4696 - accuracy: 0.7969
Epoch 46/100
28/28 [==============================] - 0s 938us/step - loss: 0.4802 - accuracy: 0.7991
Epoch 47/100
28/28 [==============================] - 0s 631us/step - loss: 0.4775 - accuracy: 0.7969
Epoch 48/100
28/28 [==============================] - 0s 918us/step - loss: 0.4703 - accuracy: 0.79